In [1]:
import requests
import json

In [17]:
r=requests.post("http://127.0.0.1:5000/query",data=json.dumps({"msg":"वडिलांचा नाव कार्डवर चुकीचा आहे"}), headers={"Content-Type":"application/json"})
r

<Response [500]>

In [13]:
r.content.decode("iso-8859-1")

'{\n  "documents": [\n    {\n      "answer": "\\u0924\\u0941\\u092e\\u091a\\u094d\\u092f\\u093e \\u0935\\u093f\\u092d\\u093e\\u0917\\u093e\\u0924\\u0940\\u0932 \\u0935\\u093f\\u0926\\u094d\\u092f\\u093e\\u0930\\u094d\\u0925\\u0940 \\u0935\\u093f\\u092d\\u093e\\u0917\\u093e\\u091a\\u094d\\u092f\\u093e \\u0915\\u093e\\u0930\\u094d\\u092f\\u093e\\u0932\\u092f\\u093e\\u0924 ERP \\u092e\\u0927\\u094d\\u092f\\u0947 \\u0917\\u094d\\u0930\\u0947\\u0921 \\u0915\\u093e\\u0930\\u094d\\u0921/PDC \\u0935\\u0930 \\u0928\\u093e\\u0935 \\u0926\\u0941\\u0930\\u0941\\u0938\\u094d\\u0924\\u0940 \\u0915\\u0930\\u0923\\u0947 \\u0906\\u0935\\u0936\\u094d\\u092f\\u0915 \\u0906\\u0939\\u0947. ERP \\u092e\\u0927\\u094d\\u092f\\u0947 \\u0906\\u0935\\u0936\\u094d\\u092f\\u0915 \\u0926\\u0941\\u0930\\u0941\\u0938\\u094d\\u0924\\u094d\\u092f\\u093e \\u0915\\u0930\\u0923\\u094d\\u092f\\u093e\\u0938\\u093e\\u0920\\u0940 \\u0915\\u0943\\u092a\\u092f\\u093e \\u0935\\u093f\\u0926\\u094d\\u092f\\u093e\\u0930\\u094d\\u09

In [ ]:
# make a dictionary of the response
r_dict = json.loads(r.text)

In [20]:
r_dict["answers"][0]["meta"]["answer"]

' This message occurs because your result is withheld due to non-submission of eligibility documents, outstanding fees, etc. You need to contact student section for further clarification.'

In [25]:
from haystack.nodes import FARMReader
import pandas as pd
from haystack.document_stores import InMemoryDocumentStore
from haystack.nodes import TfidfRetriever
from haystack.pipelines import ExtractiveQAPipeline
from flask import Flask, make_response, request
from googletrans import Translator
import requests
from flask_cors import CORS, cross_origin

In [26]:
document_store = InMemoryDocumentStore()
retriever = TfidfRetriever(document_store=document_store)
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)
df = pd.read_csv("faq.csv")
df.fillna(value="", inplace=True)
df["question"] = df["question"].apply(lambda x: x.strip())
print(df.head())
questions = list(df["question"].values)
df = df.rename(columns={"question": "content"})
docs_to_index = df.to_dict(orient="records")
document_store.write_documents(docs_to_index)
pipe = ExtractiveQAPipeline(reader, retriever)


INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.nodes.retriever.sparse -  Found 0 candidate paragraphs from 0 docs in DB
INFO - haystack.nodes.retriever.sparse -  Found 0 candidate paragraphs from 0 docs in DB
WARNING - haystack.nodes.retriever.sparse -  Fit method called with empty document store
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/roberta-base-squad2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded deepset/roberta-base-squad2
INFO - haystack.modeling.logger -  ML Logging is turned off. No parameters, metrics or artifac

                                                                          question  \
0                                          What if I need to know my correct CGPA?   
1  I have cleared my backlog but the result sheet does not show GPA and CGPA. Why?   
2  I have cleared my backlog, but the result on ERP is not updated .It still sh...   
3                               What do I do if I have not received my grade card?   
4  My name/Father's Name/Mother's name is incorrect on grade card. How do I get...   

                                                                            answer  \
0  CGPA is seen on the grade card. The CGPA count can be verified by referring ...   
1   GPA and CGPA are not mentioned on backlog grade cards. GPA, CGPA will be hi...   
2   Grade card shows first attempt GPA and CGPA only. Hence, even if you have c...   
3  Grade cards should be collected from the student section of your Department....   
4   Name correction on grade cards/PDC needs to be do

In [27]:
prediction = pipe.run(
    query="father name wrong",
    params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}},
)

WARNING - haystack.nodes.retriever.sparse -  Indexed documents have been updated and fit() method needs to be run before retrieval. Running it now.
INFO - haystack.nodes.retriever.sparse -  Found 90 candidate paragraphs from 90 docs in DB
Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/home/jxtin/py39/lib/python3.9/site-packages/haystack/modeling/model/prediction_head.py:485: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 20.51 Batches/s]


In [35]:
prediction["documents"][0].content

"My name/Father's Name/Mother's name is incorrect on grade card. How do I get it corrected?"

In [12]:
print(r.text)

<!doctype html>
<html lang=en>
  <head>
    <title>httpcore._exceptions.ReadTimeout: The read operation timed out
 // Werkzeug Debugger</title>
    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css">
    <link rel="shortcut icon"
        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">
    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>
    <script>
      var CONSOLE_MODE = false,
          EVALEX = true,
          EVALEX_TRUSTED = false,
          SECRET = "UbKWypzwaGI8wZnfxCcw";
    </script>
  </head>
  <body style="background-color: #fff">
    <div class="debugger">
<h1>ReadTimeout</h1>
<div class="detail">
  <p class="errormsg">httpcore._exceptions.ReadTimeout: The read operation timed out
</p>
</div>
<h2 class="traceback">Traceback <em>(most recent call last)</em></h2>
<div class="traceback">
  <h3></h3>
  <ul><li><div class="frame" id="frame-140043761983152">
  <h4>File <cite class="filename">"/home/jxtin/py3

In [16]:
"वडिलांचा नाव कार्डवर चुकीचा आहे"

'वडिलांचा नाव कार्डवर चुकीचा आहे'

In [18]:
u"\u0906\u092a\u0915\u0947 \u0935\u093f\u092d\u093e\u0917 \u0915\u0947 \u091b\u093e\u0924\u094d\u0930 \u0905\u0928\u0941\u092d\u093e\u0917 \u0915\u0947 \u0915\u093e\u0930\u094d\u092f\u093e\u0932\u092f \u092e\u0947\u0902 \u0908\u0906\u0930\u092a\u0940 \u092e\u0947\u0902 \u0917\u094d\u0930\u0947\u0921 \u0915\u093e\u0930\u094d\u0921/\u092a\u0940\u0921\u0940\u0938\u0940 \u092a\u0930 \u0928\u093e\u092e \u0938\u0941\u0927\u093e\u0930 \u0915\u0930\u0928\u0947 \u0915\u0940 \u0906\u0935\u0936\u094d\u092f\u0915\u0924\u093e \u0939\u0948\u0964 \u0908\u0906\u0930\u092a\u0940 \u092e\u0947\u0902 \u0906\u0935\u0936\u094d\u092f\u0915 \u0938\u0941\u0927\u093e\u0930 \u0915\u0930\u0928\u0947 \u0915\u0947 \u0932\u093f\u090f \u0915\u0943\u092a\u092f\u093e \u091b\u093e\u0924\u094d\u0930 \u0905\u0928\u0941\u092d\u093e\u0917 \u0915\u0947 \u0915\u093e\u0930\u094d\u092f\u093e\u0932\u092f \u0938\u0947 \u0938\u0902\u092a\u0930\u094d\u0915 \u0915\u0930\u0947\u0902\u0964 \u0907\u0938\u0915\u0947 \u0905\u0932\u093e\u0935\u093e, \u091b\u093e\u0924\u094d\u0930 \u0915\u094b \u0905\u092a\u0928\u0947 \u0908\u0906\u0930\u092a\u0940 \u0932\u0949\u0917\u093f\u0928 \u092e\u0947\u0902 \u0938\u092e\u092f-\u0938\u092e\u092f \u092a\u0930 \u0905\u092a\u0928\u0947 \u0935\u094d\u092f\u0915\u094d\u0924\u093f\u0917\u0924 \u0935\u093f\u0935\u0930\u0923 \u0915\u0940 \u091c\u093e\u0902\u091a \u0915\u0930\u0928\u0940 \u091a\u093e\u0939\u093f\u090f\u0964"

'आपके विभाग के छात्र अनुभाग के कार्यालय में ईआरपी में ग्रेड कार्ड/पीडीसी पर नाम सुधार करने की आवश्यकता है। ईआरपी में आवश्यक सुधार करने के लिए कृपया छात्र अनुभाग के कार्यालय से संपर्क करें। इसके अलावा, छात्र को अपने ईआरपी लॉगिन में समय-समय पर अपने व्यक्तिगत विवरण की जांच करनी चाहिए।'